In [1]:
from datetime import date
from db_connectors import *

def get_query_from_df(schema, table, df):
    query_list = []
    df = df.fillna('NULL')
    for index, row in df.iterrows():
        row = row.apply(lambda x: x.replace("'",'`') if type(x) == str else 
                        (str(x) if type(x) == pd._libs.tslibs.timestamps.Timestamp or type(x) == date else float(x)))
        query_list.append(str(tuple(row)).replace("'NULL'", "NULL").replace("''", "NULL"))

    query_full = f'INSERT INTO {schema}.{table} VALUES' +  ', '.join(query_list)
    return query_full

In [2]:
# загружаем файл
df = pd.read_excel('kmd.xlsx',sheet_name='kmd')
df.head(5)

Business Area \n(Бизнес область) Organisation Domain  \
0                Sales and Service           WHOLESALE   
1                Sales and Service           WHOLESALE   
2                Sales and Service           WHOLESALE   
3                Sales and Service           WHOLESALE   
4                Sales and Service           WHOLESALE   

  Ответственный на уровне домена Functional Area  \
0                    Иевлев А.В.      Marketing    
1                    Иевлев А.В.       Marketing   
2                    Иевлев А.В.           Sales   
3                    Иевлев А.В.           Sales   
4                    Иевлев А.В.           Sales   

  ABE (Aggregated Business Entry)\nName + ABE  \
0                      Promo Events Corp ABE    
1                     Client Surveys Corp ABE   
2                         Sales Plan Corp ABE   
3                       Underwriting Corp ABE   
4                      Client Offers Corp ABE   

  ABE (Aggregated Business Entry)\nName                Описание ABE  \
0                    Promo Events Corp   Маркетинговые акции для ЮЛ   
1                   Client Surveys Corp          Опросы клиентов ЮЛ   
2                       Sales Plan Corp     План по продажам для ЮЛ   
3                     Underwriting Corp             Андеррайтинг ЮЛ   
4                    Client Offers Corp      Предложения клиенту ЮЛ   

   BE (Business Entry)  Описание BE  Источник сущности  \
0                   NaN         NaN  DPA Marts, BIAN,    
1                   NaN         NaN               BIAN   
2                   NaN         NaN           GP, BIAN   
3                   NaN         NaN        BIAN, ADOIT   
4                   NaN         NaN          DPA Marts   

                              Департамент                Business Actor  \
0          Департамент кредитных операций    Кисляков Сергей Евгеньевич   
1                                     NaN                           NaN   
2          Департамент кредитных операций    Кисляков Сергей Евгеньевич   
3          Департамент кредитных операций    Кисляков Сергей Евгеньевич   
4  Департамент взаимодействия с клиентами  Ахапкина Татьяна Анатольевна   

                     Бизнес процесс  Unnamed: 13  Unnamed: 14  Unnamed: 15  \
0                               NaN          NaN          NaN          NaN   
1                               NaN          NaN          NaN          NaN   
2  A.02.05 Кредитование клиентов ЮЛ          NaN          NaN          NaN   
3                               NaN          NaN          NaN          NaN   
4                               NaN          NaN          NaN          NaN   

   Unnamed: 16  Unnamed: 17  
0          NaN          NaN  
1          NaN          NaN  
2          NaN          NaN  
3          NaN          NaN  
4          NaN          NaN

In [3]:
df.rename(columns={'Functional Area': 'fa',
                  'ABE (Aggregated Business Entry)\nName':'abe',
                   'Описание ABE':'abe_description',
                   'Organisation Domain':'organisation_domain'
                  }, inplace=True)

In [4]:
df = df[['organisation_domain','fa','abe','abe_description']].query('abe == abe')
df

organisation_domain            fa                  abe  \
0             WHOLESALE    Marketing    Promo Events Corp    
1             WHOLESALE     Marketing  Client Surveys Corp   
2             WHOLESALE         Sales      Sales Plan Corp   
3             WHOLESALE         Sales    Underwriting Corp   
4             WHOLESALE         Sales   Client Offers Corp   
..                  ...           ...                  ...   
383             FINANCE       Finance  Financial reporting   
384            SECURITY      SECURITY   Security Assurance   
385            SECURITY      SECURITY             Messages   
386            SECURITY      SECURITY      Asset insurance   
387                DATA  Data Quality        Data Glossary   

                                       abe_description  
0                           Маркетинговые акции для ЮЛ  
1                                   Опросы клиентов ЮЛ  
2                              План по продажам для ЮЛ  
3                                      Андеррайтинг ЮЛ  
4                               Предложения клиенту ЮЛ  
..                                                 ...  
383                              Финансовая отчетность  
384  Вирусная угроза, выявленная в процессе проверк...  
385  Почтовое сообщение с уведомлением результата п...  
386                               Страхование активов   
387                                   Глоссарий данных  

[388 rows x 4 columns]

In [5]:
# удалим лишнее

for i in df.columns:
    df[i] = df[i].str.strip().str.lower().str.replace('/', ' ').str.replace('-', ' ').str.replace('/s+', ' ', regex=True)

In [6]:
df

organisation_domain            fa                  abe  \
0             wholesale     marketing    promo events corp   
1             wholesale     marketing  client surveys corp   
2             wholesale         sales      sales plan corp   
3             wholesale         sales    underwriting corp   
4             wholesale         sales   client offers corp   
..                  ...           ...                  ...   
383             finance       finance  financial reporting   
384            security      security   security assurance   
385            security      security             messages   
386            security      security      asset insurance   
387                data  data quality        data glossary   

                                       abe_description  
0                           маркетинговые акции для юл  
1                                   опросы клиентов юл  
2                              план по продажам для юл  
3                                      андеррайтинг юл  
4                               предложения клиенту юл  
..                                                 ...  
383                              финансовая отчетность  
384  вирусная угроза, выявленная в процессе проверк...  
385  почтовое сообщение с уведомлением результата п...  
386                                страхование активов  
387                                   глоссарий данных  

[388 rows x 4 columns]

In [7]:
# проверка на вшивость
if len(df['abe'].str.contains(r'[a-z ]').loc[df['abe'].str.contains(r'[^-0-9a-z ]')].index) > 0:
    print('abe', df['abe'].str.contains(r'[a-z ]').loc[df['abe'].str.contains(r'[^-0-9a-z ]')].index)
else:
    print('abe - норм')
    
if len(df['fa'].str.contains(r'[a-z ]').loc[df['fa'].str.contains(r'[^-0-9a-z ]')].index) > 0:
    print('fa', df['fa'].str.contains(r'[a-z ]').loc[df['fa'].str.contains(r'[^-0-9a-z ]')].index)
else:
    print('fa - норм')
    
if len(df['organisation_domain'].str.contains(r'[a-z ]').loc[df['organisation_domain'].str.contains(r'[^-0-9a-z ]')].index) > 0:
    print('organisation_domain', df['organisation_domain'].str.contains(r'[a-z ]').loc[df['organisation_domain'].str.contains(r'[^-0-9a-z ]')].index)
else:
    print('organisation_domain - норм')

abe - норм
fa - норм
organisation_domain - норм


# Заливка ABE

In [8]:
# execute_dq_profiling("""
# DROP TABLE IF EXISTS cmd.abe cascade """)


# execute_dq_profiling("""
# CREATE TABLE cmd.abe (
#     id_abe SERIAL PRIMARY KEY,
#     abe VARCHAR UNIQUE NOT NULL,
#     abe_description varchar NULL,
#     organisation_domain varchar NOT NULL,
#     abe_owner varchar NULL
# )
# """)

# execute_dq_profiling("""
# SET search_path TO cmd;
# ALTER SEQUENCE abe_id_abe_seq RESTART WITH 1000;
# """)

In [9]:
# проверка на базу

In [10]:
# проверка на дубли описаний

if len(df[['abe']].drop_duplicates()) != len(df[['abe','abe_description']].drop_duplicates()):
    double_description = list(df[['abe','abe_description']].drop_duplicates().groupby('abe').count().query('abe_description > 1').index)
    print('Дубли в файле !!!!!')
    print(df[['abe','abe_description']].drop_duplicates().query('abe in @double_description'))
else:
    double_description = []
    print('Дублей нет')

Дубли в файле !!!!!
                        abe           abe_description
100     loyalty program vip  программа лояльности vip
103     loyalty program vip         priority pass vip
151  loyalty program retail   программы лояльности фл
152        bank card retail       банковская карта фл
153        bank card retail              овердрафт фл
154  loyalty program retail          priority pass фл


In [11]:
# проверка на дубли доменов
if len(df[['abe']].drop_duplicates()) != len(df[['abe','abe_description']].drop_duplicates()):
    double_domain = list(df[['organisation_domain','abe']].drop_duplicates().groupby('abe').count().query('organisation_domain > 1').index)
    print('Дубли в файле !!!!!')
    print(df[['organisation_domain','abe']].drop_duplicates().query('abe in @double_domain'))
else:
    double_domain = []
    print('Дублей нет')


Дубли в файле !!!!!
    organisation_domain            abe
260       finmonitoring  risks metrics
267               risks  risks metrics


In [12]:
# Очистим abe от дублей
double_list = list(set(double_domain)| set(double_description))

df_double = df[['abe','abe_description','organisation_domain']].drop_duplicates().query('abe in @double_list')
df = df.query('abe not in @double_list')

In [13]:
# заливка
query = get_query_from_df('cmd','abe',df[['abe','abe_description','organisation_domain']].drop_duplicates())
execute_dq_profiling(query.replace('INSERT INTO cmd.abe', 'INSERT INTO cmd.abe (abe, abe_description, organisation_domain) '))

# Заливка FA

In [14]:
# execute_dq_profiling("""DROP TABLE IF EXISTS cmd.fa cascade""")

# execute_dq_profiling("""
# CREATE TABLE cmd.fa (
#     id_fa SERIAL PRIMARY KEY,
#     fa VARCHAR UNIQUE NOT NULL
# );
# """)

# execute_dq_profiling("""
# SET search_path TO cmd;
# ALTER SEQUENCE fa_id_fa_seq RESTART WITH 1000;
# """)

In [15]:
# проверка на базу

In [16]:
# заливка
query = get_query_from_df('cmd','fa',df[['fa']].drop_duplicates())
execute_dq_profiling(query.replace('INSERT INTO cmd.fa', 'INSERT INTO cmd.fa (fa) ').replace(',)', ')'))

# Таблица связей

In [17]:
df_fa = get_df_from_dq_profiling(""" SELECT * FROM cmd.fa """)
df_abe = get_df_from_dq_profiling(""" SELECT id_abe, abe FROM cmd.abe """)

In [18]:
df_key = pd.merge(
    df[['fa','abe']].drop_duplicates(), 
    df_fa, 
    on=['fa'],
    how='left')

df_key = pd.merge(
    df_key, 
    df_abe, 
    on=['abe'],
    how='left')

In [19]:
df_key[['id_fa','id_abe']]

id_fa  id_abe
0     1000    1000
1     1000    1001
2     1001    1002
3     1001    1003
4     1001    1004
..     ...     ...
266   1037    1263
267   1038    1264
268   1038    1265
269   1038    1266
270   1039    1267

[271 rows x 2 columns]

In [21]:
# execute_dq_profiling("""DROP TABLE IF EXISTS cmd.abe_fa cascade""")

# execute_dq_profiling("""
# CREATE TABLE cmd.abe_fa (
#     id_abe integer REFERENCES cmd.abe (id_abe),
#     id_fa integer REFERENCES cmd.fa (id_fa)
# );
# """)

# execute_dq_profiling("""
# SET search_path TO cmd;
# ALTER TABLE abe_fa
# ADD CONSTRAINT uc_abe_fa UNIQUE (id_abe ,id_fa) """)

In [23]:
# проверка на базу

In [24]:
# заливка
query = get_query_from_df('cmd','abe_fa',df_key[['id_abe','id_fa']].drop_duplicates())
execute_dq_profiling(query)

In [25]:
# Дубли

# list(df_key[['id_abe','id_fa']].drop_duplicates().groupby('id_abe').count().query('id_fa > 1').index)

In [26]:
# execute_dq_profiling("""
# CREATE OR REPLACE VIEW cmd.entity_relationships
# AS SELECT  
#     'FA.'||af.id_fa as ID_FA, 
#     'ABE.'||af.id_abe as ID_ABE,
#     initcap(f.fa) as FA, 
#     initcap(a.abe) as ABE,
#     initcap(a.abe_description) as abe_description,
#     upper(a.organisation_domain) as organisation_domain,
#     initcap(a.abe_owner) as abe_owner
# FROM cmd.abe_fa as af
# LEFT JOIN cmd.abe  as a
#     on af.id_abe = a.id_abe
# LEFT JOIN cmd.fa  as f
#     on af.id_fa = f.id_fa
# order by f.fa, a.abe
# """)

# Дубли

In [27]:
df_double

abe           abe_description organisation_domain
100     loyalty program vip  программа лояльности vip                 vip
103     loyalty program vip         priority pass vip                 vip
151  loyalty program retail   программы лояльности фл              retail
152        bank card retail       банковская карта фл              retail
153        bank card retail              овердрафт фл              retail
154  loyalty program retail          priority pass фл              retail
260           risks metrics              риск метрики       finmonitoring
267           risks metrics              риск метрики               risks